<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np, matplotlib.pyplot
from astropy.io import fits
import pysynphot as pyS
import pcmdpy as ppy

GPU acceleration not available, sorry
No module named 'pycuda'


In [45]:
def compute_zpts(instrument, detector, band, mjd):
    bandpass = pyS.ObsBandpass('{:s},{:s},{:s},mjd#{:d}'.format(instrument, detector, band, mjd))
    spec_bb = pyS.BlackBody(50000)
    spec_bb_norm = spec_bb.renorm(1, 'counts', bandpass)
    obs = pyS.Observation(spec_bb_norm, bandpass)
    zps = {}
    zps['vega'] = obs.effstim('vegamag')
    zps['st'] = obs.effstim('stmag')
    zps['ab'] = obs.effstim('abmag')
    return zps

In [59]:
def process_fits(file_name):
    with fits.open(file_name) as hdu:
        header = hdu[0].header
    instrument = header['INSTRUME'].lower().strip(' ')
    detector = header['DETECTOR'].lower().strip(' ')
    if detector == 'wfc':
        detector = 'wfc1'
    band = None
    for k in ['FILTER', 'FILTER1', 'FILTER2']:
        if k in header:
            b_temp = header[k]
            if 'CLEAR' in b_temp:
                continue
            else:
                band = b_temp
                break
    if band is None:
        raise KeyError('Unable to identify filter from FITS file')
    mjd = int(header['PROCTIME'])
    exposure = header['EXPTIME']
    zpts = compute_zpts(instrument, detector, band, mjd)
    print('Filter: {:s}'.format(band))
    print('Observation Date: {:d} (MJD)'.format(mjd))
    print('Vega ZeroPoint: {:.4f}'.format(zpts['vega']))
    print('AB ZeroPoint: {:.4f}'.format(zpts['ab']))
    print('ST ZeroPoint: {:.4f}'.format(zpts['st']))
    print('Exposure Time: {:.1f}'.format(exposure))

In [60]:
process_fits('../../data/h_m51_b_s05_drz_sci.fits')

Filter: F435W
Observation Date: 53478 (MJD)
Vega ZeroPoint: 25.7885
AB ZeroPoint: 25.6903
ST ZeroPoint: 25.1801
Exposure Time: 16320.0


In [63]:
process_fits('../../data/hlsp_phat_hst_acs-wfc_12105-m31-b06-f02_f475w_v1_drz.fits')

Filter: F475W
Observation Date: 56126 (MJD)
Vega ZeroPoint: 26.1511
AB ZeroPoint: 26.0586
ST ZeroPoint: 25.7483
Exposure Time: 1710.0


In [61]:
process_fits('../../data/h_m51_v_s05_drz_sci.fits')

Filter: F555W
Observation Date: 53478 (MJD)
Vega ZeroPoint: 25.7318
AB ZeroPoint: 25.7319
ST ZeroPoint: 25.6857
Exposure Time: 8160.0


In [66]:
print('M51 Observations')
process_fits('../../data/h_m51_i_s05_drz_sci.fits')

M51 Observations
Filter: F814W
Observation Date: 53478 (MJD)
Vega ZeroPoint: 25.5283
AB ZeroPoint: 25.9565
ST ZeroPoint: 26.7927
Exposure Time: 8160.0


In [65]:
print('M31 Observations with PHAT')
process_fits('../../data/hlsp_phat_hst_acs-wfc_12105-m31-b06-f02_f814w_v1_drz.fits')

M31 Observations with PHAT
Filter: F814W
Observation Date: 56126 (MJD)
Vega ZeroPoint: 25.5191
AB ZeroPoint: 25.9471
ST ZeroPoint: 26.7827
Exposure Time: 1505.0
